In [ ]:
import numpy as np
import pandas as pd
import scipy.stats as st

from statsmodels.stats import proportion, weightstats
from statsmodels.stats.proportion import proportions_ztest
from math import ceil
from IPython.display import display, Latex

# Reference

> [Unit: Two-sample inference for the difference between groups](https://www.khanacademy.org/math/statistics-probability/significance-tests-confidence-intervals-two-samples)<br>
> [Test statistic](https://en.wikipedia.org/wiki/Test_statistic)

---

# Two-proportion z test

> [TO POOL OR NOT TO POOL](https://www.ijpam.eu/contents/2013-89-4/5/5.pdf): There are two versions of this test, one is used when the variances of the two populations are equal (the pooled test) and the other one is used when the variances of the two populations are unequal (the unpooled test).

---

## Pooled z test

### Formula

$\displaystyle H_{0}\colon p_{1}=p_{2} \Rightarrow d_0 = 0$

$\displaystyle z={\frac {({\hat {p}}_{1}-{\hat {p}}_{2})}{\sqrt {{\hat {p}}(1-{\hat {p}})({\frac {1}{n_{1}}}+{\frac {1}{n_{2}}})}}}$

$\displaystyle \hat{p}=\frac{x_1 + x_2}{n_1 + n_2}$

### Conditions

- **Random**
- **Normal**
    - $n_1 p_1 > 5$ **and** $n_1(1 − p_1) > 5$
    - $n_2 p_2 > 5$ **and** $n_2(1 − p_2) > 5$
- **Independent**

---

### Example 1

- $H_0: p_1 - p_2 = 0 \Rightarrow P(\hat p_1 - \hat p_2 | H_0) < 5\%$
- $H_\text{a}: p_1 - p_2 \neq 0$
- $\alpha = 5\%$

- $k1 = 642, n1 = 1000$
- $k2 = 591, n1 = 1000$

In [ ]:
# Two-sample pooled z-test for proportion manually
k1, n1 = 642, 1000
k2, n2 = 591, 1000
alpha = .05
CL = 1 - alpha

# estimated proportion
p_hat_1 = k1 / n1
p_hat_2 = k2 / n2
p_hat = (k1 + k2) / (n1 + n2)

# difference
d0 = 0
d = p_hat_1 - p_hat_2

# standard error
SE = np.sqrt(p_hat*(1-p_hat) * (1/n1 + 1/n2))

# margin of error
critical_value = st.norm.interval(CL)[1]
MOE = critical_value * SE

# confidence interval
ci_low, ci_upp = d - MOE, d + MOE
print('CI:', (ci_low, ci_upp))

# z statistic & p-value
zstat = (d - d0) / SE  # pooled
if zstat > 0:
    pval = st.norm.sf(zstat) * 2
else:
    pval = st.norm.cdf(zstat) * 2
print('zstat, pval:', (zstat, pval))

In [ ]:
# Two-sample pooled z-test for proportion by proportions_ztest
k1, n1 = 642, 1000
k2, n2 = 591, 1000
d0 = 0
count = [k1, k2]
nobs = [n1, n2]

zstat, pval = proportions_ztest(
    count, 
    nobs,
    value=d0,  # null hypothesis
    alternative='two-sided', 
    prop_var=False  # pooled
)
print('zstat, pval:', (zstat, pval))

In [ ]:
# R Chi-square test
from rpy2.robjects.packages import importr
import rpy2.robjects as robjects

k1, n1 = 642, 1000
k2, n2 = 591, 1000
alpha = .05
CL = 1 - alpha

stats = importr("stats")
result = stats.prop_test(
    x = robjects.IntVector([k1, k2]), 
    n = robjects.IntVector([n1, n2]),
    alternative = 'two.sided',
    conf_level = CL,
    correct = False  # True if either the expected successes or failures is < 5
)
print(result)

---

## Unpooled z test

### Formula

$\displaystyle H_{0}\colon |d_0| > 0$

$\displaystyle z=\frac{(\hat{p}_1 - \hat{p}_2) - d_0}{\sqrt{\frac{\hat{p}_1(1 - \hat{p}_1)}{n_1} + \frac{\hat{p}_2(1 - \hat{p}_2)}{n_2}}}$

### Conditions

- **Random**
- **Normal**
    - $n_1 p_1 > 5$ **and** $n_1(1 − p_1) > 5$
    - $n_2 p_2 > 5$ **and** $n_2(1 − p_2) > 5$
- **Independent**

---

### Example 1

- $H_0: p_1 - p_2 = 0 \Rightarrow P(\hat p_1 - \hat p_2 | H_0) < 5\%$
- $H_\text{a}: p_1 - p_2 \neq 0$
- $\alpha = 5\%$

- $k1 = 642, n1 = 1000$
- $k2 = 591, n1 = 1000$

In [ ]:
k1, n1 = 642, 1000
k2, n2 = 591, 1000
alpha = .05
CL = 1 - alpha

# estimated proportion
p_hat_1 = k1 / n1
p_hat_2 = k2 / n2
p_hat = (k1 + k2) / (n1 + n2)

# difference
d0 = 0
d = p_hat_1 - p_hat_2

var1 = p_hat_1 * (1 - p_hat_1) / n1
var2 = p_hat_2 * (1 - p_hat_2) / n2

# standard error
SE = np.sqrt(var1 + var2)

# margin of error
critical_value = st.norm.interval(CL)[1]
MOE = critical_value * SE

# confidence interval
ci_low, ci_upp = d - MOE, d + MOE
print('CI:', (ci_low, ci_upp))

# z statistic & p-value
# zstat = (d - d0) / np.sqrt(p_hat * (1-p_hat) * (1/n1 + 1/n2)) # pooled
zstat = (d - d0) / SE # unpooled
pval = st.norm(loc=d0, scale=SE).sf(x=abs(d)) * 2 # unpooled
# OR
# pval = st.norm.sf(zstat) * 2 # pooled or unpooled depends on zstat
print('zstat, pval:', (zstat, pval))

In [ ]:
# I modifed the function from https://www.statsmodels.org/dev/_modules/statsmodels/stats/proportion.html#proportions_ztest
def proportions_ztest(count, nobs, value=None, alternative='two-sided',
                      prop_var=False):
    """
    Test for proportions based on normal (z) test

    Parameters
    ----------
    count : {int, array_like}
        the number of successes in nobs trials. If this is array_like, then
        the assumption is that this represents the number of successes for
        each independent sample
    nobs : {int, array_like}
        the number of trials or observations, with the same length as
        count.
    value : float, array_like or None, optional
        This is the value of the null hypothesis equal to the proportion in the
        case of a one sample test. In the case of a two-sample test, the
        null hypothesis is that prop[0] - prop[1] = value, where prop is the
        proportion in the two samples. If not provided value = 0 and the null
        is prop[0] = prop[1]
    alternative : str in ['two-sided', 'smaller', 'larger']
        The alternative hypothesis can be either two-sided or one of the one-
        sided tests, smaller means that the alternative hypothesis is
        ``prop < value`` and larger means ``prop > value``. In the two sample
        test, smaller means that the alternative hypothesis is ``p1 < p2`` and
        larger means ``p1 > p2`` where ``p1`` is the proportion of the first
        sample and ``p2`` of the second one.
    prop_var : False or float in (0, 1)
        If prop_var is false, then the variance of the proportion estimate is
        calculated based on the sample proportion. Alternatively, a proportion
        can be specified to calculate this variance. Common use case is to
        use the proportion under the Null hypothesis to specify the variance
        of the proportion estimate.

    Returns
    -------
    zstat : float
        test statistic for the z-test
    p-value : float
        p-value for the z-test

    Examples
    --------
    >>> count = 5
    >>> nobs = 83
    >>> value = .05
    >>> stat, pval = proportions_ztest(count, nobs, value)
    >>> print('{0:0.3f}'.format(pval))
    0.695

    >>> import numpy as np
    >>> from statsmodels.stats.proportion import proportions_ztest
    >>> count = np.array([5, 12])
    >>> nobs = np.array([83, 99])
    >>> stat, pval = proportions_ztest(count, nobs)
    >>> print('{0:0.3f}'.format(pval))
    0.159

    Notes
    -----
    This uses a simple normal test for proportions. It should be the same as
    running the mean z-test on the data encoded 1 for event and 0 for no event
    so that the sum corresponds to the count.

    In the one and two sample cases with two-sided alternative, this test
    produces the same p-value as ``proportions_chisquare``, since the
    chisquare is the distribution of the square of a standard normal
    distribution.
    """
    # TODO: verify that this really holds
    # TODO: add continuity correction or other improvements for small samples
    # TODO: change options similar to propotion_ztost ?

    count = np.asarray(count)
    nobs = np.asarray(nobs)

    if nobs.size == 1:
        nobs = nobs * np.ones_like(count)

    prop = count * 1. / nobs
    k_sample = np.size(prop)
    if value is None:
        if k_sample == 1:
            raise ValueError('value must be provided for a 1-sample test')
        value = 0
    if k_sample == 1:
        diff = prop - value
    elif k_sample == 2:
        diff = prop[0] - prop[1] - value
    else:
        msg = 'more than two samples are not implemented yet'
        raise NotImplementedError(msg)

    # unpooled
    if prop_var:
        var_ = np.sum((count / nobs) * (1 - (count / nobs)) / nobs)
    # pooled
    else:
        p_pooled = np.sum(count) * 1. / np.sum(nobs)
        nobs_fact = np.sum(1. / nobs)
        var_ = p_pooled * (1 - p_pooled) * nobs_fact
    std_diff = np.sqrt(var_)

    from statsmodels.stats.weightstats import _zstat_generic2
    return _zstat_generic2(diff, std_diff, alternative)

In [ ]:
# Two-sample unpooled z-test for proportion by proportions_ztest
k1, n1 = 642, 1000
k2, n2 = 591, 1000
d0 = 0
count = [k1, k2]
nobs = [n1, n2]

zstat, pval = proportions_ztest(
    count, 
    nobs,
    value=d0,  # null hypothesis
    alternative='two-sided', 
    prop_var=True  # unpooled
)
print('zstat, pval:', (zstat, pval))

In [ ]:
# R Chi-square test
from rpy2.robjects.packages import importr
import rpy2.robjects as robjects

k1, n1 = 642, 1000
k2, n2 = 591, 1000
alpha = .05
CL = 1 - alpha

stats = importr("stats")
result = stats.prop_test(
    x = robjects.IntVector([k1, k2]), 
    n = robjects.IntVector([n1, n2]),
    alternative = 'two.sided',
    conf_level = CL,
    correct = False # True if either the expected successes or failures is < 5
)
print(result)

---

# Two-sample t test

> [Student's t-test](https://en.wikipedia.org/wiki/Student%27s_t-test#Dependent_t-test_for_paired_samples)

- Paired(dependent) t-test: Since we are ultimately concerned with the difference between two measures in one sample, the paired t-test reduces to the one sample t-test.
- Unpaired(independent) t-test
    - pooled t-test
    - unpooled t-test

---

## Paired t test

---

### Example 1: Two-sided

> [How to Conduct a Paired Samples T-Test in Python](https://www.geeksforgeeks.org/how-to-conduct-a-paired-samples-t-test-in-python/)

Let us consider that we want to know whether an engine oil significantly impacts the car’s mileage of different brands. In order to test this, we have 10 cars in a garage doped with original engine oil initially. We have noted their mileage for $100$ kilometers each. Then, we have each of the cars doped with another engine oil (different from the original one). Then, the mileage of the cars is calculated for 100 kilometers each. To compare the difference between the mean mileage of the first and second test, we use a paired samples t-test because for each car their first test score can be paired with their second test score. Conducting paired sample T-test is a step-by-step process.

In [ ]:
# pre holds the mileage before 
# applying the different engine oil
pre = np.array([30, 31, 34, 40, 36, 35, 34, 30, 28, 29])
  
# post holds the mileage after 
# applying the different engine oil
post = np.array([30, 31, 32, 38, 32, 31, 32, 29, 28, 30])

In [ ]:
# Performing the paired sample t-test
st.ttest_rel(a=pre, b=post, alternative='two-sided')

In [ ]:
# OR simplify the paired t test to one sample t test
diff = post - pre
n, mu_0, mu_1, sd_1 = len(diff), 0, diff.mean(), diff.std(ddof=1)
SE = sd_1 / np.sqrt(n)

if diff.mean() < 0:
    pval = st.t(loc=mu_0, scale=SE, df=n-1).cdf(x=mu_1) * 2
else:
    pval = st.t(loc=mu_0, scale=SE, df=n-1).sf(x=mu_1) * 2
pval

---

## Pooled t test

---

### Example 1: Two-sided

> [How to Conduct a Two Sample T-Test in Python](https://www.statology.org/two-sample-t-test-python/)

Researchers want to know whether or not two different species of plants have the same mean height. To test this, they collect a simple random sample of 20 plants from each species.

In [ ]:
A = np.array([14, 15, 15, 16, 13, 8, 14, 17, 16, 14, 19, 20, 21, 15, 15, 16, 16, 13, 14, 12])
B = np.array([15, 17, 14, 17, 14, 8, 12, 19, 19, 14, 17, 22, 24, 16, 13, 16, 13, 18, 15, 13])

In [ ]:
# t statistic & p-value
st.ttest_ind(
    a = A, 
    b = B, 
    equal_var = True, # True to perform pooled t test; False to perform unpooled t test
    alternative='two-sided'
)

In [ ]:
# OR
weightstats.ttest_ind(
    x1 = A, 
    x2 = B,
    alternative = 'two-sided',
    usevar='pooled',
)

---

## Unpooled t test

---

### Example 1: Two-sided

> [How to Conduct a Two Sample T-Test in Python](https://www.statology.org/two-sample-t-test-python/)

Researchers want to know whether or not two different species of plants have the same mean height. To test this, they collect a simple random sample of 20 plants from each species.

In [ ]:
A = np.array([14, 15, 15, 16, 13, 8, 14, 17, 16, 14, 19, 20, 21, 15, 15, 16, 16, 13, 14, 12])
B = np.array([15, 17, 14, 17, 14, 8, 12, 19, 19, 14, 17, 22, 24, 16, 13, 16, 13, 18, 15, 13])

In [ ]:
# t statistic & p-value
st.ttest_ind(
    a = A, 
    b = B, 
    equal_var = False, # True to perform pooled t test; False to perform unpooled t test
    alternative='two-sided'
)

In [ ]:
# OR
weightstats.ttest_ind(
    x1 = A, 
    x2 = B,
    alternative = 'two-sided',
    usevar='unequal',
)